In [ ]:
!sudo apt update
!sudo apt install gcc


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,031 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,396 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,598 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,372 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,162 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,200 kB]
Get:13 https:

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving CrackAZ99.c to CrackAZ99.c
Saving EncryptSHA512.c to EncryptSHA512.c


In [2]:
!gcc EncryptSHA512.c -o encrypt -lcrypt


In [3]:
!./encrypt HP93

$6$AS$Ig.vW9RG9J5gPFUvHwyV67GdVVndF.2ROH6.qZjQN1Nm5kqn0t/FKNf4.48qRHdyAWwIQOtKkCosTrwyj3SvJ.


In [6]:
%%writefile CrackAZ99.c
#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include <crypt.h>
#include <pthread.h>
#include <unistd.h>

int count = 0; // Global count of explored combinations
int found = 0; // Flag to stop threads once password is found
pthread_mutex_t lock;

void substr(char *dest, char *src, int start, int length) {
    memcpy(dest, src + start, length);
    *(dest + length) = '\0';
}

typedef struct {
    char *salt_and_encrypted;
    char start;
    char end;
} ThreadData;

void *crack_thread(void *arg) {
    ThreadData *data = (ThreadData *) arg;
    int x, y, z;
    char salt[7];
    char plain[7];
    char *enc;

    substr(salt, data->salt_and_encrypted, 0, 6);

    for (x = data->start; x <= data->end && !found; x++) {
        for (y = 'A'; y <= 'Z' && !found; y++) {
            for (z = 0; z <= 99 && !found; z++) {
                sprintf(plain, "%c%c%02d", x, y, z);
                enc = (char *) crypt(plain, salt);
                pthread_mutex_lock(&lock);
                count++;
                pthread_mutex_unlock(&lock);
                if (strcmp(data->salt_and_encrypted, enc) == 0) {
                    pthread_mutex_lock(&lock);
                    found = 1;  // Set found flag to true
                    printf("#%-8d%s %s\n", count, plain, enc);
                    pthread_mutex_unlock(&lock);
                    return NULL;
                }
            }
        }
    }

    return NULL;
}

int main(int argc, char *argv[]) {
    if (argc < 2) {
        printf("Usage: %s <encrypted password>\n", argv[0]);
        return 1;
    }

    pthread_t threads[2];
    ThreadData thread_data[2];
    pthread_mutex_init(&lock, NULL);

    // First thread will crack from A to M
    thread_data[0].salt_and_encrypted = argv[1];
    thread_data[0].start = 'A';
    thread_data[0].end = 'M';

    // Second thread will crack from N to Z
    thread_data[1].salt_and_encrypted = argv[1];
    thread_data[1].start = 'N';
    thread_data[1].end = 'Z';

    // Create threads
    pthread_create(&threads[0], NULL, crack_thread, &thread_data[0]);
    pthread_create(&threads[1], NULL, crack_thread, &thread_data[1]);

    // Wait for threads to finish
    pthread_join(threads[0], NULL);
    pthread_join(threads[1], NULL);

    pthread_mutex_destroy(&lock);

    printf("%d solutions explored\n", count);

    return 0;
}

Overwriting CrackAZ99.c


In [7]:
!gcc -pthread CrackAZ99.c -o crack -lcrypt



In [8]:
!./crack '$6$AS$Ig.vW9RG9J5gPFUvHwyV67GdVVndF.2ROH6.qZjQN1Nm5kqn0t/FKNf4.48qRHdyAWwIQOtKkCosTrwyj3SvJ.'


67600 solutions explored
